<a href="https://colab.research.google.com/github/cangokalp/char_rnn_rap_generator/blob/master/char_rnn_rap_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/cangokalp/char_rnn_rap_generator/blob/master/char_rnn_rap_generator.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

In [0]:
import pdb
import pickle
import re

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    IS_COLAB = True
except Exception:
    IS_COLAB = False

import tensorflow as tf
from tensorflow import keras
import sklearn
import numpy as np
import os
import sys

In [0]:
#Go to my github https://github.com/cangokalp/char_rnn_rap_generator
#and download the two files; lyrics_array_mix.pickle - under data folder.
#After that run the below cell. 
#Then on the tab to the left of the screen there is a folder icon, 
#click that and youll see data folder
#click upload and upload the files you downloaded. 
#Then drag them in the data folder
#then you can continue

In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')

# DRIVE_PATH = '/content/gdrive/My Drive'

In [0]:
PROJECT_ROOT_DIR = "."
DATA_PATH = os.path.join(PROJECT_ROOT_DIR, "data")
os.makedirs(DATA_PATH, exist_ok=True)

MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "saved_models")
os.makedirs(MODEL_PATH, exist_ok=True)

def save(fname, data, extension="pickle"):
    path = os.path.join(DATA_PATH, fname + "." + extension)
    with open(path, 'wb') as f:
        pickle.dump(data, f)

def load(fname, extension='pickle'):
    path = os.path.join(DATA_PATH, fname + "." + extension)
    
    with open(path, 'rb') as f:
        item = pickle.load(f)

    return item

In [0]:
#GRAB TRAINING DATA:
documents = load('lyrics_array_mix')

In [82]:
#CREATE NEW TRAINING DATA - IF YOU WANT

import time
!pip install lyricsgenius
import lyricsgenius

API_TOKEN = '1OWmIr37ZuUI5hDXjN_FIXJlbZadIqH6JBDObBCiF_MNySn4BQqh8aQ8OUYF5muk'
fname = 'mix'

#### grab the lyrics #####
artists = []
genius = lyricsgenius.Genius(API_TOKEN, skip_non_songs=True, remove_section_headers=True)

# lonely = genius.search_artist("The Lonely Island", max_songs=80, sort="popularity")
# artists.append(lonely)
# time.sleep(30)

fetty = genius.search_artist("Fetty Wap", max_songs=15, sort="popularity")
artists.append(fetty)
childish = genius.search_artist("Childish Gambino", max_songs=20, sort="popularity")
artists.append(childish)
time.sleep(10)

drake = genius.search_artist("Drake", max_songs=20, sort="popularity")
artists.append(drake)
time.sleep(10)
kendrick = genius.search_artist("Kendrick Lamar", max_songs=20, sort="popularity")
artists.append(kendrick)
time.sleep(20)

# dicky = genius.search_artist("Lil Dicky", max_songs=20, sort="popularity")
# artists.append(dicky)

travis = genius.search_artist("Travis Scott", max_songs=10, sort="popularity")
artists.append(travis)
q = genius.search_artist("ScHoolboy Q", max_songs=10, sort="popularity")
artists.append(q)
wayne = genius.search_artist("Lil Wayne", max_songs=15, sort="popularity")
artists.append(wayne)
time.sleep(30)

eminem = genius.search_artist("Eminem", max_songs=10, sort="popularity")
artists.append(eminem)
ariana = genius.search_artist("Ariana Grande", max_songs=5, sort="popularity")
artists.append(ariana)
wiz = genius.search_artist("Wiz Khalifa", max_songs=10, sort="popularity")
artists.append(wiz)
billie = genius.search_artist("Billie Eilish", max_songs=15, sort="popularity")
artists.append(billie)
peas = genius.search_artist("Black Eyed Peas", max_songs=10, sort="popularity")
artists.append(peas)
time.sleep(40)

chance = genius.search_artist("Chance The Rapper", max_songs=10, sort="popularity")
artists.append(chance)
kanye = genius.search_artist("Kanye West", max_songs=15, sort="popularity")
artists.append(kanye)
outkast = genius.search_artist("OutKast", max_songs=10, sort="popularity")
artists.append(outkast)
rihanna = genius.search_artist("Rihanna", max_songs=10, sort="popularity")
artists.append(rihanna)


documents = []
for artist in artists:
  for song in artist.songs: 
    documents.append(song.lyrics)

save('lyrics_array_' + fname, documents)

Searching for songs by Fetty Wap...

Song 1: "Trap Queen"
Song 2: "679"
Song 3: "My Way (Remix)"
Song 4: "My Way"
Song 5: "Again"
Song 6: "Jimmy Choo"
Song 7: "RGF Island"
Song 8: "Jugg"
Song 9: "D.A.M (Dats All Me)"
Song 10: "Wake Up"
Song 11: "No Days Off"
Song 12: "Trap Niggas Freestyle"
Song 13: "Time"
Song 14: "ZooWap"
Song 15: "How We Do Things"

Reached user-specified song limit (15).
Done. Found 15 songs.
Searching for songs by Childish Gambino...

Song 1: "This Is America"
Song 2: "Redbone"
Song 3: "V. 3005"
Song 4: "IV. Sweatpants"
Song 5: "Bonfire"
Song 6: "Heartbeat"
Song 7: "Freaks and Geeks"
Song 8: "III. Telegraph Ave. (”Oakland” by Lloyd)"
Song 9: "I. The Worst Guys"
Song 10: "You See Me"
Song 11: "Sober"
Song 12: "Me and Your Mama"
Song 13: "II. Zealots of Stockholm [Free Information]"
Song 14: "I. Crawl"
Song 15: "II. Worldstar"
Song 16: "Feels Like Summer"
Song 17: "III. Life: The Biggest Troll [Andrew Auernheimer]"
Song 18: "That Power"
Song 19: "L.E.S."
Song 20: "I

In [0]:
# path='/content/gdrive/My Drive/lyrics_array_mix.pickle'
# with open(path, 'wb') as f:
#     pickle.dump(documents, f)

In [0]:
# COMBINE ALL SONGS INTO ONE TEXT FILE

comedy_rap_text = ''
for document in documents:
  if document is not None:
    comedy_rap_text += document + '\n'


In [0]:
# CLEAN THE TEXT, GET RID OF SOME PATTERNS

comedy_rap_prepped = comedy_rap_text
import re
pattern = re.compile(r'[(.?)]')
comedy_rap_prepped = re.sub(pattern, '', comedy_rap_prepped)

In [52]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime → "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Wed May  6 22:41:52 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    31W / 250W |   1573MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
# rap_bot_v3 will use statefull RNN - expecting it to do better than rap_bot_v2 which used stateless RNN
# Basically in a stateful model 
# after each training batch the final state is kept and used as initial state for next batch
# this way it'll remember things beyond 100 chars
statefull = True

In [0]:
# PREPROCESSING THE TRAINING DATA SO THAT CHAR MODEL CAN USE

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(comedy_rap_prepped)
max_id = len(tokenizer.word_index) 
dataset_size = tokenizer.document_count

train_size = dataset_size * 90 // 100
n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead

[encoded] = np.array(tokenizer.texts_to_sequences([comedy_rap_prepped])) - 1

if not statefull:

  dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
  dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_length))

  np.random.seed(42)
  tf.random.set_seed(42)

  batch_size = 32
  dataset = dataset.shuffle(10000).batch(batch_size)
  dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
  dataset = dataset.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
  dataset = dataset.prefetch(1)

  for X_batch, Y_batch in dataset.take(1):
      print(X_batch.shape, Y_batch.shape)
else:

  dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
  dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
  dataset = dataset.flat_map(lambda window: window.batch(window_length))
  dataset = dataset.repeat().batch(1)
  dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
  dataset = dataset.map(
      lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
  dataset = dataset.prefetch(1)

  batch_size = 32
  encoded_parts = np.array_split(encoded[:train_size], batch_size)
  datasets = []
  for encoded_part in encoded_parts:
      dataset = tf.data.Dataset.from_tensor_slices(encoded_part)
      dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
      dataset = dataset.flat_map(lambda window: window.batch(window_length))
      datasets.append(dataset)
  dataset = tf.data.Dataset.zip(tuple(datasets)).map(lambda *windows: tf.stack(windows))
  dataset = dataset.repeat().map(lambda windows: (windows[:, :-1], windows[:, 1:]))
  dataset = dataset.map(
      lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
  dataset = dataset.prefetch(1)

In [63]:
# model = keras.models.load_model('./saved_models/rap_bot_v3_model.h5')
# model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


In [39]:
# IF YOU WANT TO TRAIN FROM SCRATCH - RNN NETWORK WITH DROPOUT
if not statefull:

  model = keras.models.Sequential([
      keras.layers.GRU(128, return_sequences=True, input_shape=[None, max_id],
                      dropout=0.2, recurrent_dropout=0.2),
      keras.layers.GRU(128, return_sequences=True,
                      dropout=0.2, recurrent_dropout=0.2),
      keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                      activation="softmax"))
  ])

else:

  model = keras.models.Sequential([
      keras.layers.GRU(256, return_sequences=True, stateful=True,
                      dropout=0.2, recurrent_dropout=0.3,
                      batch_input_shape=[batch_size, None, max_id]),
      keras.layers.GRU(256, return_sequences=True, stateful=True,
                      dropout=0.2, recurrent_dropout=0.3),
      keras.layers.GRU(256, return_sequences=True, stateful=True,
                      dropout=0.2, recurrent_dropout=0.3),
      keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                      activation="softmax"))
  ])

  class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()


model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


In [41]:
if not statefull:
  history = model.fit(dataset, steps_per_epoch=train_size // batch_size, epochs=10)
else:
  steps_per_epoch = train_size // batch_size // n_steps
  history = model.fit(dataset, steps_per_epoch=steps_per_epoch, epochs=50,
                    callbacks=[ResetStatesCallback()])

Epoch 1/50
173/173 [==============================] - 61s 354ms/step - loss: 2.6654
Epoch 2/50
173/173 [==============================] - 61s 352ms/step - loss: 2.2053
Epoch 3/50
173/173 [==============================] - 60s 348ms/step - loss: 2.0797
Epoch 4/50
173/173 [==============================] - 60s 349ms/step - loss: 2.0132
Epoch 5/50
173/173 [==============================] - 60s 346ms/step - loss: 2.4577
Epoch 6/50
173/173 [==============================] - 60s 345ms/step - loss: 2.4000
Epoch 7/50
173/173 [==============================] - 59s 343ms/step - loss: 2.5996
Epoch 8/50
173/173 [==============================] - 59s 342ms/step - loss: 2.3306
Epoch 9/50
173/173 [==============================] - 60s 345ms/step - loss: 2.3576
Epoch 10/50
173/173 [==============================] - 59s 340ms/step - loss: 2.2562
Epoch 11/50
173/173 [==============================] - 59s 341ms/step - loss: 2.2079
Epoch 12/50
173/173 [==============================] - 59s 339ms/step - lo

In [0]:
if statefull:
  stateless_model = keras.models.Sequential([
    keras.layers.GRU(256, return_sequences=True, input_shape=[None, max_id]),
    keras.layers.GRU(256, return_sequences=True),
    keras.layers.GRU(256, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                    activation="softmax"))
  ])
  stateless_model.build(tf.TensorShape([None, None, max_id]))
  stateless_model.set_weights(model.get_weights())
  model = stateless_model

In [43]:
# SAVE THE TRAINED NEW MODEL
model.save('./saved_models/rap_bot_v3_model.h5')

# from google.colab import drive
# drive.mount('/content/gdrive')
# DRIVE_PATH = '/content/gdrive/My Drive'
# model.save(DRIVE_PATH + '/rap_bot_v3_model.h5')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# HELPER FUNCTIONS FOR NEW LYRICS GENERATION

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text


In [70]:
# THIS WILL GENERATE WARNINGS - JUST IGNORE
# temperature param represents how the next word is sampled, close to 0 just does argmax, 
# around 1 worked good for me you can just try different and look at the results
# to not to keep generating same things usually they dont do argmax over the probabilities instead you sample from them with the prob of the probabilities
# n_chars - number of characters you want to generate
tf.random.set_seed(42)
text = complete_text("w", temperature=0.9, n_chars=800)

In [71]:
# !!!!! LETS SEEE :D
print(text)

where i wish it out this
vower, when you started to feel loyalty, let you all of lavino
yeah, yeah, my mind of my side
don't make you to big shit
make a long get your bitch, look out to be mine, "nigga ah!
oh, who are you was a hood, skin it lonk
we are you "driving", i'm combort to stop my drugs me started you drunk, "ooh, good change
where you world got me from busting" "you say i get another never got a couple of my bood
"where's the streets the gloads thoulder
i'm call with the legs all addicted
"let me to a cop probably pick that!"
you know what to get to eat that got bove you how to the money off night
i miss me ucational too court, but so high
but hundredies couldn't even need some back in the same going homie
where's a verse, on the worry
my light now that we got for the people
migh


In [72]:
from google.colab import drive
drive.mount('/content/gdrive')
DRIVE_PATH = '/content/gdrive/My Drive'
f = open(DRIVE_PATH + '/rap_bot_lyrics.txt', 'w+')
f.write(text)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


801